In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import classification_report 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

In [ ]:
dataX,dataY,dataCenter=make_blobs(n_samples=200,
                                  n_features=2,
                                  return_centers=True,
                                  centers=3,
                                  random_state=42,
                                  cluster_std=3.3
                                 )

In [ ]:
from scipy.stats.mstats_basic import skew
skew(dataX)
#skewness

In [ ]:
sb.set()
plt.scatter(dataX[:,0],dataX[:,1],c=dataY,cmap="viridis_r")
plt.scatter(dataCenter[:,0],dataCenter[:,1],c="red",s=55)
plt.title("RawData")

In [ ]:
kk=KMeans(random_state=106,n_clusters=3,init="k-means++",max_iter=100,verbose=1,algorithm="lloyd")
kk.fit(dataX)

In [ ]:
plt.scatter(dataX[:,0],dataX[:,1],c=kk.labels_,cmap="viridis_r")
plt.scatter(kk.cluster_centers_[:,0],kk.cluster_centers_[:,1],c="Red",s=65)
plt.title("Sklearn Prediction")

In [ ]:
import numpy as np
from scipy.spatial import distance
import random
#random.seed(42)

class KMeansAlgorithms:
    def __init__(self, k=3, max_iterations=100, use_elkan=False, use_lloyd=True):
        self.k = k #
        self.max_iterations = max_iterations
        self.use_elkan = use_elkan 
        self.use_lloyd = use_lloyd 
        
    # Initialize centroids using KMeans++
    def initialize_centroids(self, X): #Kmeans++
        centroids = []
        centroid_idx = random.choice(range(X.shape[0]))
        centroids.append(X[centroid_idx]) 
        
        distances = np.zeros((X.shape[0],))
        remaining_points = set(range(X.shape[0]))
        
        for _ in range(self.k - 1):
            for idx in remaining_points:
                distances[idx] = min([distance.euclidean(X[idx], c) for c in centroids])

            probabilities = distances / sum(distances) 
            next_centroid_idx = np.random.choice(range(X.shape[0]), p=probabilities) 
            centroids.append(X[next_centroid_idx])
            remaining_points.remove(next_centroid_idx)

        return np.array(centroids)
    
    # Assign each point to its closest cluster center
    def assign_clusters(self, X, centroids):
        clusters = np.zeros((X.shape[0],), dtype=np.int32)
        
        for i in range(len(X)):
            closest_centroid_index = np.argmin([distance.euclidean(X[i], c) for c in centroids])
            clusters[i] = closest_centroid_index
        
        return clusters
     
    # Update centroids based on assigned clusters
    def update_centroids(self, X, clusters):
        updated_centroids = np.zeros((self.k, X.shape[1]))
        counts = np.zeros((self.k, ))

        for idx, label in enumerate(clusters):
            updated_centroids[label] += X[idx]
            counts[label] += 1
            
        for i in range(updated_centroids.shape[0]):
            updated_centroids[i] /= counts[i]
        
        return updated_centroids
      
    # --->Elkan algorithm<---
    def update_centroids_elkan(self, X, clusters, old_centroids):
        updated_centroids = np.copy(old_centroids)
        changes = False

        for j in range(self.k):
            neighbor_list = list(set(range(X.shape[0])) - set(clusters[clusters != j]))
            
            squared_distances = np.zeros((len(neighbor_list)))
            
            for i, neighbor in enumerate(neighbor_list):

                sqr_dist = distance.squareform(distance.pdist([X[neighbor], updated_centroids[j]], 'euclidean'))
                lowerbound = max([sqr_dist[0][1] + sqr_dist[1][0] - distance.squareform(distance.pdist(old_centroids[[j]]))[0][0]])
                upperbound = sqr_dist[0][0]
                bounds = (lowerbound <= upperbound) | ((upperbound - lowerbound) < 1e-10)
                squared_distances[i] = upperbound if bounds else float('inf')
             
            if len(squared_distances) > 0:
                min_index = np.argmin(squared_distances)
                updated_centroids[j] = X[neighbor_list[min_index]]
                changes = True
            else:
                updates_needed = True
                while updates_needed:
                    updates_needed = False
                    neighbors = np.where(clusters == j)[0]
                    non_empty_dimensions = np.count_nonzero(~np.isnan(X[neighbors].sum(axis=0)), axis=-1)
                    dimensions_to_drop = np.argsort(non_empty_dimensions)[::-1][self.k:]
                    if len(dimensions_to_drop) > 0:
                        drop_indices = np.ix_(dimensions_to_drop, range(X.shape[1]))
                        dropped_data = np.delete(X[neighbors], dimensions_to_drop, axis=1)
                        
                        mean_dropped_data = np.mean(dropped_data, axis=0)
                        indices = np.arange(len(neighbors)).reshape(-1, 1)[:, np.newaxis]
                        means = np.repeat(mean_dropped_data[np.newaxis, :], repeats=len(neighbors), axis=0)
                        differences = X[neighbors][indices, drop_indices] - means[indices, drop_indices]
                        
                        squares = np.power(differences, 2)
                        variances = np.sum(squares, axis=1)/(len(neighbors)-1)
                        threshold = np.percentile(variances, 100-(100/(len(neighbors)+1)))
                        
                        selected_dimensions = np.where(variances > threshold)[0]
                        selected_dimensions = np.concatenate([selected_dimensions, dimensions_to_drop]).tolist()
                        X[neighbors] = X[neighbors][..., selected_dimensions]
                        
                        updated_centroids[j] = np.mean(X[neighbors], axis=0)
                        if len(selected_dimensions) < X.shape[1]:
                            updates_needed = True
                            changes = True
                        elif np.linalg.norm(updated_centroids[j] - old_centroids[j]) > 1e-10:
                            changes = True
                        else:
                            break
        
        return updated_centroids if changes else old_centroids

    # --->Fitting Function<---
    def fit(self, X):
        centroids = self.initialize_centroids(X)
        previous_centroids = None
        iteration = 0

        if self.use_elkan:
            fitting_func = self._fit_elkan
        elif self.use_lloyd:
            fitting_func = self._fit_lloyd
        else:
            raise ValueError("Please select valid fitting method.")

        while True:
            iteration += 1 
            clusters = self.assign_clusters(X, centroids)
            old_centroids = centroids
            centroids = fitting_func(X, clusters, old_centroids)
            
            if np.allclose(old_centroids, centroids):
                break
            
            if iteration >= self.max_iterations:
                break
          
        return centroids, clusters

    def _fit_elkan(self, X, clusters, old_centroids):
        centroids = self.update_centroids_elkan(X, clusters, old_centroids)
        return centroids

    def _fit_lloyd(self, X, clusters, old_centroids):
        centroids = self.update_centroids(X, clusters)
        return centroids

# Usage Example
    """kmeans = KMeansAlgorithms(k=3, max_iterations=100, use_elkan=True, use_lloyd=False)
    centroids, clusters = kmeans.fit(X)
    print("Centroids:", centroids)
    print("Clusters:", clusters)
    kmeans = KMeansAlgorithms(k=3, max_iterations=100, use_elkan=False, use_lloyd=True)
    centroids, clusters = kmeans.fit(X)
    print("Centroids:", centroids)
    print("Clusters:", clusters)"""

In [ ]:
kmaCent,kmalabel=KMeansAlgorithms(k=3,max_iterations=100,use_elkan=False,use_lloyd=True).fit(dataX)
kmaCent2,kmalabel2=KMeansAlgorithms(k=3,max_iterations=100,use_elkan=True,use_lloyd=False).fit(dataX)

In [ ]:
plt.subplots(1,2,figsize=(11,5))
plt.subplot(1,2,1)
plt.scatter(dataX[:,0],dataX[:,1],c=kmalabel,cmap="viridis_r")
plt.scatter(kmaCent[:,0],kmaCent[:,1],c="Red",s=65)
plt.title("lloyd pred")

plt.subplot(1,2,2)
plt.scatter(dataX[:,0],dataX[:,1],c=kmalabel2,cmap="viridis_r")
plt.scatter(kmaCent2[:,0],kmaCent2[:,1],c="Red",s=65)
plt.title("Elkan pred")

In [ ]:
#KMeans Sklearn
confusion_matrix(dataY,kk.labels_)

In [ ]:
#made-up kmeans elkan algo
confusion_matrix(dataY,kmalabel2)

In [ ]:
#made-up kmeans lloyd algo
confusion_matrix(dataY,kmalabel)

In [ ]:
print(f"Statistics for Sklearn:\n{classification_report(dataY,kk.labels_)}\n")

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
kk_params={"init":["k-means++","random"],#2 
           "copy_x":[True,False],#2 
           "tol":[1e-5,1e-4,1e-3,1e-2,1e-2,0.1],#6 
           "algorithm":["lloyd","elkan"],#2 
          "max_iter":[50,100,200,300]#4
          }
kkgrd=GridSearchCV(estimator=KMeans(n_clusters=3,random_state=42),param_grid=kk_params)
kkgrd.fit(dataX)

In [ ]:
kkgrd.best_params_

In [ ]:
confusion_matrix(dataY,KMeans(n_clusters=3,max_iter=50,copy_x=True,init="random",
                              tol=0.1,algorithm="lloyd",random_state=42).fit_predict(dataX)

In [ ]:
#madeup-lloyd:6
#sklearn-lloyd:11

In [ ]:
import time
start=time.perf_counter_ns()
kk=KMeans(random_state=106,n_clusters=3,init="k-means++",max_iter=100,algorithm="lloyd")
print(f"{time.perf_counter_ns()-start:,} ns")

In [ ]:
start=time.perf_counter_ns()
kk2=KMeansAlgorithms(k=3,max_iterations=100,use_elkan=False,use_lloyd=True)
print(f"{time.perf_counter_ns()-start:,} ns")